[Covertype dataset](https://archive.ics.uci.edu/dataset/31/covertype) on UCI ML Repository

In [27]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

### Read the dataset

In [28]:
colnames = ["Elevation", "Aspect", "Slope", \
"Horizontal_Distance_To_Hydrology", \
"Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways",
"Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm", \
"Horizontal_Distance_To_Fire_Points"] + \
[f"Wilderness_Area_{i}" for i in range(4)] + \
[f"Soil_Type_{i}" for i in range(40)] + \
["Cover_Type"]

# alternatively, colnames = df.schema.names

In [48]:
df = spark.read.csv("data/covertype/covtype.data.gz", header=False, inferSchema=True)
df = df.toDF(*colnames)

df = df.na.drop() # handle missing values lol

### Split into testing and training data

In [30]:
train_data, test_data = df.randomSplit([0.9, 0.1])
train_data.cache()
test_data.cache()

DataFrame[Elevation: int, Aspect: int, Slope: int, Horizontal_Distance_To_Hydrology: int, Vertical_Distance_To_Hydrology: int, Horizontal_Distance_To_Roadways: int, Hillshade_9am: int, Hillshade_Noon: int, Hillshade_3pm: int, Horizontal_Distance_To_Fire_Points: int, Wilderness_Area_0: int, Wilderness_Area_1: int, Wilderness_Area_2: int, Wilderness_Area_3: int, Soil_Type_0: int, Soil_Type_1: int, Soil_Type_2: int, Soil_Type_3: int, Soil_Type_4: int, Soil_Type_5: int, Soil_Type_6: int, Soil_Type_7: int, Soil_Type_8: int, Soil_Type_9: int, Soil_Type_10: int, Soil_Type_11: int, Soil_Type_12: int, Soil_Type_13: int, Soil_Type_14: int, Soil_Type_15: int, Soil_Type_16: int, Soil_Type_17: int, Soil_Type_18: int, Soil_Type_19: int, Soil_Type_20: int, Soil_Type_21: int, Soil_Type_22: int, Soil_Type_23: int, Soil_Type_24: int, Soil_Type_25: int, Soil_Type_26: int, Soil_Type_27: int, Soil_Type_28: int, Soil_Type_29: int, Soil_Type_30: int, Soil_Type_31: int, Soil_Type_32: int, Soil_Type_33: int, S

### Collapse all those columns into a "feature vector"

In [31]:
from pyspark.ml.feature import VectorAssembler

input_cols = colnames[:-1]
vector_assembler = VectorAssembler(inputCols=input_cols, outputCol="featureVector")

assembled_train_data = vector_assembler.transform(train_data)
assembled_test_data = vector_assembler.transform(test_data)

assembled_train_data.select("featureVector").show(3, truncate=False)

+--------------------------------------------------------------------------------------------------+
|featureVector                                                                                     |
+--------------------------------------------------------------------------------------------------+
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1859.0,18.0,12.0,67.0,11.0,90.0,211.0,215.0,139.0,792.0,1.0,1.0])|
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1860.0,18.0,13.0,95.0,15.0,90.0,210.0,213.0,138.0,780.0,1.0,1.0])|
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1861.0,35.0,14.0,60.0,11.0,85.0,218.0,209.0,124.0,832.0,1.0,1.0])|
+--------------------------------------------------------------------------------------------------+
only showing top 3 rows



### Train the model

In [34]:
from pyspark.ml.classification import DecisionTreeClassifier

classifier = DecisionTreeClassifier(seed=1234, labelCol=colnames[-1], featuresCol="featureVector", 
    predictionCol="prediction")

model = classifier.fit(assembled_train_data)
model.toDebugString

'DecisionTreeClassificationModel: uid=DecisionTreeClassifier_c11bda6ebb9d, depth=5, numNodes=43, numClasses=8, numFeatures=54\n  If (feature 0 <= 3047.5)\n   If (feature 0 <= 2561.5)\n    If (feature 10 <= 0.5)\n     If (feature 0 <= 2450.5)\n      If (feature 3 <= 15.0)\n       Predict: 4.0\n      Else (feature 3 > 15.0)\n       Predict: 3.0\n     Else (feature 0 > 2450.5)\n      If (feature 17 <= 0.5)\n       Predict: 2.0\n      Else (feature 17 > 0.5)\n       Predict: 3.0\n    Else (feature 10 > 0.5)\n     Predict: 2.0\n   Else (feature 0 > 2561.5)\n    If (feature 0 <= 2952.5)\n     If (feature 15 <= 0.5)\n      If (feature 17 <= 0.5)\n       Predict: 2.0\n      Else (feature 17 > 0.5)\n       Predict: 3.0\n     Else (feature 15 > 0.5)\n      Predict: 3.0\n    Else (feature 0 > 2952.5)\n     If (feature 3 <= 191.0)\n      If (feature 36 <= 0.5)\n       Predict: 2.0\n      Else (feature 36 > 0.5)\n       Predict: 1.0\n     Else (feature 3 > 191.0)\n      Predict: 2.0\n  Else (featur

### Evaluate

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

predictions = model.transform(assembled_test_data)

evaluator = MulticlassClassificationEvaluator(labelCol=colnames[-1], predictionCol="prediction")
accuracy = evaluator.setMetricName("accuracy").evaluate(predictions)
f1 = evaluator.setMetricName("f1").evaluate(predictions)

print("accuracy", accuracy)
print("f1", f1)

accuracy 0.7043966323666978
f1 0.6863404644444491


In [ ]:
confusion_matrix = predictions.groupBy(colnames[-1]) \
    .pivot("prediction") \
    .count() \
    .na.fill(0) \
    .orderBy(colnames[-1])

confusion_matrix.show()

+----------+-----+-----+----+---+----+
|Cover_Type|  1.0|  2.0| 3.0|4.0| 7.0|
+----------+-----+-----+----+---+----+
|         1|14553| 6006|  23|  0| 413|
|         2| 5489|22021| 469| 12|  52|
|         3|    0|  683|2899| 60|   0|
|         4|    0|    1| 171|128|   0|
|         5|    1|  900| 101|  0|   0|
|         6|    0|  660| 986| 74|   0|
|         7|  952|    2|   9|  0|1061|
+----------+-----+-----+----+---+----+



### Hyperparameter Tuning

Not in manual again.

In [38]:
from pyspark.ml import Pipeline

assembler = VectorAssembler(inputCols=colnames[:-1], outputCol="featureVector")
classifier = DecisionTreeClassifier(featuresCol="featureVector", labelCol=colnames[-1], predictionCol="prediction")
pipeline = Pipeline(stages=[assembler, classifier])

`TrainValidationSplit` is like `CrossValidator`, but it performs the split only once. Good for speed.

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

grid = ParamGridBuilder() \
    .addGrid(classifier.impurity, ["gini", "entropy"]) \
    .addGrid(classifier.maxDepth, [1, 20]) \
    .addGrid(classifier.maxBins, [40, 300]) \
    .addGrid(classifier.minInfoGain, [0.0, 0.05]) \
    .build()

evaluator = MulticlassClassificationEvaluator(labelCol="Cover_Type", predictionCol="prediction", 
    metricName="accuracy")

validator = TrainValidationSplit(estimator=pipeline, evaluator=evaluator, 
    estimatorParamMaps=grid)
validator_model = validator.fit(train_data)

In [46]:
params = validator_model.getEstimatorParamMaps() # list of param maps for all models
metrics = validator_model.validationMetrics 

metrics_and_params = list(zip(metrics, params))
metrics_and_params.sort(key=lambda x: x[0], reverse=True)
metrics_and_params[0]

(0.9111124696244899,
 {Param(parent='DecisionTreeClassifier_b94ed8101630', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'entropy',
  Param(parent='DecisionTreeClassifier_b94ed8101630', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 20,
  Param(parent='DecisionTreeClassifier_b94ed8101630', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 300,
  Param(parent='DecisionTreeClassifier_b94ed8101630', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0})

In [47]:
evaluator.evaluate(validator_model.bestModel.transform(test_data))

24/11/08 15:16:27 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB


0.908788385606152

The usage for `RandomForestClassifier` is the exact same. Even the hyperparameters are the same.